In [ ]:
import threading  # Importa a biblioteca threading para executar a máquina de Turing em paralelo

class TuringMachine:
    def __init__(self, tape, start_state='q0'):
        """
        Inicializa a máquina de Turing com duas fitas e dois cabeçotes de leitura.

        :param tape: A entrada inicial na fita.
        :param start_state: O estado inicial da máquina.
        """
        self.tape1 = list(tape)  # Primeira fita
        self.tape2 = list(tape)  # Segunda fita
        self.head_position1 = 0  # Posição do cabeçote na primeira fita
        self.head_position2 = 0  # Posição do cabeçote na segunda fita
        self.state1 = start_state  # Estado atual da primeira fita
        self.state2 = start_state  # Estado atual da segunda fita
        self.accepted1 = False  # Flag para verificar se a primeira fita aceitou a entrada
        self.accepted2 = False  # Flag para verificar se a segunda fita aceitou a entrada

        # Regras de transição para a primeira fita (verificação de palíndromo)
        self.transitions1 = {
            ('q0', '0'): ('q0', '0', 'R'),
            ('q0', '1'): ('q1', '1', 'R'),
            ('q0', ' '): ('q_accept', ' ', 'R'),
            ('q1', '0'): ('q2', '0', 'R'),
            ('q1', '1'): ('q0', '1', 'R'),
            ('q2', '0'): ('q1', '0', 'R'),
            ('q2', '1'): ('q2', '1', 'R'),
        }

        # Regras de transição para a segunda fita (verificação de divisibilidade por 3)
        self.transitions2 = {
            ('q0', '0'): ('q1', ' ', 'R'),
            ('q0', '1'): ('q4', ' ', 'R'),
            ('q0', ' '): ('q_accept', ' ', 'R'),
            ('q1', '0'): ('q1', '0', 'R'),
            ('q1', '1'): ('q1', '1', 'R'),
            ('q1', ' '): ('q2', ' ', 'L'),
            ('q2', '0'): ('q3', ' ', 'L'),
            ('q2', ' '): ('q0', ' ', 'R'),
            ('q3', '0'): ('q3', '0', 'L'),
            ('q3', '1'): ('q3', '1', 'L'),
            ('q3', ' '): ('q0', ' ', 'R'),
            ('q4', '0'): ('q4', '0', 'R'),
            ('q4', '1'): ('q4', '1', 'R'),
            ('q4', ' '): ('q5', ' ', 'L'),
            ('q5', '1'): ('q6', ' ', 'L'),
            ('q5', ' '): ('q0', ' ', 'R'),
            ('q6', '0'): ('q6', '0', 'L'),
            ('q6', '1'): ('q6', '1', 'L'),
            ('q6', ' '): ('q0', ' ', 'R'),
        }

    def step_1(self):
        """
        Executa um passo da máquina de Turing na primeira fita.
        Verifica se atingiu o estado de aceitação ('q_accept').
        """
        if self.state1 == 'q_accept':
            self.accepted1 = True
            return False  # Para a execução

        current_symbol = self.tape1[self.head_position1]  # Lê o símbolo atual
        transition = self.transitions1.get((self.state1, current_symbol))  # Obtém a transição

        if not transition:
            return False  # Para a execução se não houver transição válida

        new_state, new_symbol, move_direction = transition  # Define a nova transição
        self.tape1[self.head_position1] = new_symbol  # Escreve o novo símbolo na fita
        self.state1 = new_state  # Atualiza o estado
        self.head_position1 += 1 if move_direction == 'R' else -1  # Move o cabeçote
        return True

    def step_2(self):
        """
        Executa um passo da máquina de Turing na segunda fita.
        Verifica se atingiu o estado de aceitação ('q_accept').
        """
        if self.state2 == 'q_accept':
            self.accepted2 = True
            return False  # Para a execução

        current_symbol = self.tape2[self.head_position2]  # Lê o símbolo atual
        transition = self.transitions2.get((self.state2, current_symbol))  # Obtém a transição

        if not transition:
            return False  # Para a execução se não houver transição válida

        new_state, new_symbol, move_direction = transition  # Define a nova transição
        self.tape2[self.head_position2] = new_symbol  # Escreve o novo símbolo na fita
        self.state2 = new_state  # Atualiza o estado
        self.head_position2 += 1 if move_direction == 'R' else -1  # Move o cabeçote
        return True

    def run_1(self):
        """Executa a primeira fita até que ela chegue a um estado de aceitação ou falhe."""
        while not self.accepted1:
            if not self.step_1():
                break

    def run_2(self):
        """Executa a segunda fita até que ela chegue a um estado de aceitação ou falhe."""
        while not self.accepted2:
            if not self.step_2():
                break

    def run_parallel(self):
        """
        Executa as duas máquinas de Turing simultaneamente em threads separadas.
        """
        thread1 = threading.Thread(target=self.run_1)  # Cria a primeira thread
        thread2 = threading.Thread(target=self.run_2)  # Cria a segunda thread

        thread1.start()  # Inicia a primeira máquina
        thread2.start()  # Inicia a segunda máquina

        thread1.join()  # Aguarda a finalização da primeira máquina
        thread2.join()  # Aguarda a finalização da segunda máquina

        return self.accepted1 and self.accepted2  # Ambas precisam aceitar para ser verdadeiro

# Teste da máquina de Turing com um número
numeros_teste = ['1001']  # Lista de números para testar
for numero in numeros_teste:
    input_tape = numero + " "  # Adiciona um espaço no final da entrada (para a fita)
    tm = TuringMachine(input_tape)  # Inicializa a máquina de Turing
    if tm.run_parallel():  # Executa as máquinas em paralelo
        print(f"O {numero} é palíndromo e divisível por 3")
    else:
        print(f"O {numero} não é palíndromo ou não é divisível por 3")


O 1001 é palíndromo e divisível por 3
